In [1]:
#Import Libraries
import os
import gc
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import logging
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import metrics
import gensim.models.keyedvectors as word2vec
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D,LSTM,GRU
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.initializers import *
from keras.optimizers import Adam, Adadelta
from keras.callbacks import Callback
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

Using TensorFlow backend.


In [2]:
import nltk
import re, string
from nltk.tokenize import TweetTokenizer   
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()
eng_stopwords = set(stopwords.words("english"))

In [3]:
#data file
train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')
submission = pd.read_csv('../input/quora-insincere-questions-classification/sample_submission.csv')

In [4]:
train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [5]:
#count the occourence of a word in whole document
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab
vocab_text = pd.concat([train['question_text'],test['question_text']])
vocab = build_vocab(vocab_text)
vocablist = list()
for i in vocab.keys():
    if vocab[i] > 100:
        vocablist.append(i)
misspellword = pd.DataFrame(vocablist)
misspellword.to_csv('misspellword.csv')

In [6]:
misspell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 
                'counselling': 'counseling', 'theatre': 'theater','cancelled': 'canceled', 'labour': 'labor', 
                'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 
                'Qoura': 'Quora', 'sallary': 'salary','Whta': 'What', 'wht':'what', 'whatis':'what is','narcisist': 'narcissist',
                'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many',
                'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 
                'banglore': 'bengaluru','mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota','obamacare': 'obama care', 
                'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 
                'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization',
                'cannot': 'can not','doesnt': 'does not','ignou': 'ignore','aiims': 'aims','barack': 'barack obama','jong-un': 'kim jong un',
                 'atleast': 'at least','iam': 'i am','ww2': 'world war 2','ww1': 'world war 1', 'arya': 'area', 'diarrhea': 'diarrhoea', 'po': 'pop'
               }

In [7]:
# PREPROCESSING PART
fill = {
        "ain't": "is not", "aren't": "are not","can't": "cannot","can't've": "cannot have", "'cause": "because", "could've": "could have", 
        "couldn't": "could not", "couldn't've": "could not have","didn't": "did not","doesn't": "does not", "don't": "do not", "hadn't": "had not", 
        "hadn't've": "had not have","i’ve":"i have", "hasn't": "has not", "haven't": "have not","he'd": "he would", "he'd've": "he would have", "he'll": "he will",
        "he'll've": "he he will have", "he's": "he is", "how'd": "how did","how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
        "I'd": "I would", "I'd've": "I would have", "I'll": "I will","I'll've": "I will have","I'm": "I am", "I've": "I have","i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
        "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
        "it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us", "ma'am": "madam", "mayn't": "may not", 
        "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
        "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
        "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
        "she's": "she is", "should've": "should have", "shouldn't": "should not","shouldn't've": "should not have", "so've": "so have","so's": "so as", 
        "this's": "this is","that'd": "that would", "that'd've": "that would have","that's": "that is","there'd": "there would", "there'd've": "there would have","there's": "there is", 
        "they'd": "they would", "they'd've": "they would have", "they'll": "they will","they'll've": "they will have", "they're": "they are", "they've": "they have", 
        "to've": "to have", "wasn't": "was not", "we'd": "we would","we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
        "we're": "we are", "we've": "we have", "weren't": "were not","what'll": "what will", "what'll've": "what will have", "what're": "what are", 
        "what's": "what is", "what've": "what have", "when's": "when is","when've": "when have", "where'd": "where did", "where's": "where is", 
        "where've": "where have", "who'll": "who will", "who'll've": "who will have","who's": "who is", "who've": "who have", "why's": "why is", 
        "why've": "why have", "will've": "will have", "won't": "will not","won't've": "will not have", "would've": "would have", "wouldn't": "would not","wouldn't've": "would not have",
        "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would",
         "you'd've": "you would have", "you'll": "you will","you'll've": "you will have", "you're": "you are", "you've": "you have",
         "children's": "childrens","women's":"womens","jehovah's":"jehovahs","girlfriend's":"girlfriends","girl's":"girls","sister's": "sisters",
          "obama's":"obamas","trump’s":"trumps","earth's":"earths","modi's":"modis","google's":"google","mcdonald's":"mcdonalds"
      }

In [8]:
def clean_text(text):    
    #fixing apostrope
    text = text.replace("’", "'")
    #to lower
    text = text.lower() #convert all of the question text to lower case
    #remove \n
    text = re.sub("\\n","",text)
    #remove url
    text = re.sub(r'http\S+', '', text)
    
    
    #Split the sentences into words
    words = tokenizer.tokenize(text)
    words = [fill[word] if word in fill else word for word in words]
    words = [misspell_dict[word] if word in misspell_dict else word for word in words]
    #words = [lem.lemmatize(word, "v") for word in words]
    words = [w for w in words if not w in eng_stopwords]
    text = " ".join(words)
    return text

In [9]:
# Clean the text
train["question_text"] = train["question_text"].apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_text(x))
#train['word_count'] = train['question_text'].apply(lambda x: len(str(x).split(" ")))

In [10]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 231806# how many unique words to use
maxlen = 72 # max number of words in a question to use

In [11]:
#fill the missing value
X_train = train["question_text"].fillna("fillna").values
y_train = train["target"].values
X_test = test["question_text"].fillna("fillna").values

In [12]:
#sentence tokenization
tokenizer = Tokenizer(num_words=max_features,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(list(X_train) + list(X_test))

In [13]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 231806 unique tokens.


In [14]:
def load_para(word_index):
    EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))#distribute the value with mean and deviation
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    del embeddings_index
    gc.collect()
    return embedding_matrix

In [15]:
def load_glove(word_index):
    EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std() #take the mean and std of embedding matrix
    embed_size = all_embs.shape[1]
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))#distribute the value with mean and deviation
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    del embeddings_index
    gc.collect()
    return embedding_matrix

In [16]:
embedding_matrix_1 = load_glove(word_index)
embedding_matrix_2 = load_para(word_index)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


In [17]:
embedding_matrix = np.mean((embedding_matrix_1, embedding_matrix_2), axis=0)
print(embedding_matrix.shape)

(231806, 300)


In [18]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [19]:
CONTEXT_DIM = 72
class Attention(Layer):

    def __init__(self, regularizer=regularizers.l2(1e-10), **kwargs):
        self.regularizer = regularizer
        self.supports_masking = True
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3        
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[-1], CONTEXT_DIM),
                                 initializer='normal',
                                 trainable=True, 
                                 regularizer=self.regularizer)
        self.b = self.add_weight(name='b',
                                 shape=(CONTEXT_DIM,),
                                 initializer='normal',
                                 trainable=True, 
                                 regularizer=self.regularizer)
        self.u = self.add_weight(name='u',
                                 shape=(CONTEXT_DIM,),
                                 initializer='normal',
                                 trainable=True, 
                                 regularizer=self.regularizer)        
        super(Attention, self).build(input_shape)

    @staticmethod
    def softmax(x, dim):
        """Computes softmax along a specified dim. Keras currently lacks this feature.
        """
        if K.backend() == 'tensorflow':
            import tensorflow as tf
            return tf.nn.softmax(x, dim)
        else:
            raise ValueError("Backend '{}' not supported".format(K.backend()))

    def call(self, x, mask=None):
        ut = K.tanh(K.bias_add(K.dot(x, self.W), self.b)) * self.u

        # Collapse `attention_dims` to 1. This indicates the weight for each time_step.
        ut = K.sum(ut, axis=-1, keepdims=True)

        # Convert those weights into a distribution but along time axis.
        # i.e., sum of alphas along `time_steps` axis should be 1.
        self.at = self.softmax(ut, dim=1)
        if mask is not None:
            self.at *= K.cast(K.expand_dims(mask, -1), K.floatx())

        # Weighted sum along `time_steps` axis.
        return K.sum(x * self.at, axis=-2)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def get_config(self):
        config = {}
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_mask(self, inputs, mask):
        return None

In [20]:
def build_model(lr = 0.0, lr_d = 0.0, units = [128,64],filters=[128,64,32], dr = 0.0):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Conv1D(filters=filters[0], kernel_size = 3, kernel_initializer='he_normal',activation='relu',padding = "valid")(x)
    x = Conv1D(filters=filters[1], kernel_size = 3,kernel_initializer='he_normal',activation='relu',padding = "valid")(x)
    x = Bidirectional(CuDNNGRU(units[0], return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(units[1], return_sequences=True))(x)
    x = Attention()(x)
    x = Dense(filters[2], kernel_initializer='he_normal',activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr = lr, decay = lr_d), metrics=['accuracy'])
    return model

In [21]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [22]:
def f1_vs_thresh(y_pred,y_val):
    thresholds = []
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        res = metrics.f1_score(y_val, (y_pred > thresh).astype(int))
        thresholds.append([thresh, res])
        print("F1 score at threshold {0} is {1}".format(thresh, res))

    thresholds.sort(key=lambda x: x[1], reverse=True)
    best_thresh = thresholds[0][0]
    print("Best threshold: ", best_thresh)
    return best_thresh

In [23]:
NFOLDS = 5
folds = KFold(n_splits=NFOLDS,random_state=10, shuffle=True)
foldscore = []
splits = folds.split(x_train, y_train)
y_test = np.zeros(x_test.shape[0])
y_oof = np.zeros(x_train.shape[0])
epochs = [8, 7, 6, 5, 4]
val_list = []

In [24]:
for fold_n, (train_index, valid_index) in enumerate(splits):
    val_list += list(valid_index)
    print('FOLD%s'%(fold_n+1))
    x_tra, x_val, y_tra, y_val = x_train[train_index], x_train[valid_index], y_train[train_index], y_train[valid_index]
    file_path = "best_model.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                  save_best_only = True, mode = "min")
    ra_val = RocAucEvaluation(validation_data=(x_val, y_val), interval = 1)
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
    callbacks = [ra_val, check_point, early_stop]
    model = build_model(lr = 1e-3, lr_d = 0, units = [128,64],filters=[128,64,32],dr = 0.2)
    print('Build Model')
    model.fit(x_tra, y_tra, batch_size=512, epochs=epochs[fold_n], 
              validation_data=(x_val, y_val), verbose=1, callbacks=callbacks 
             )
    print("train logloss:%s"%model.history.history['loss'])
    print("val logloss:%s"%model.history.history['val_loss'])
    y_pred = model.predict([x_val], batch_size=1024, verbose=2)
    y_test += np.squeeze(model.predict([x_test], batch_size = 1024, verbose = 2))/5
    y_oof[valid_index] = np.squeeze(y_pred)
    threshold = f1_vs_thresh(np.squeeze(y_pred),np.squeeze(y_val))
    foldscore.append(threshold)
pred_thresh = f1_vs_thresh(np.squeeze(y_oof[val_list]),np.squeeze(y_train[val_list]))

FOLD1
Build Model
Train on 1044897 samples, validate on 261225 samples
Epoch 1/8
1044897/1044897 [==============================] - 107s 102us/step - loss: 0.1218 - accuracy: 0.9518 - val_loss: 0.1134 - val_accuracy: 0.9551

 ROC-AUC - epoch: 1 - score: 0.953860

Epoch 00001: val_loss improved from inf to 0.11337, saving model to best_model.hdf5
Epoch 2/8
1044897/1044897 [==============================] - 104s 100us/step - loss: 0.1077 - accuracy: 0.9568 - val_loss: 0.1117 - val_accuracy: 0.9559

 ROC-AUC - epoch: 2 - score: 0.956995

Epoch 00002: val_loss improved from 0.11337 to 0.11174, saving model to best_model.hdf5
Epoch 3/8
1044897/1044897 [==============================] - 103s 99us/step - loss: 0.1007 - accuracy: 0.9594 - val_loss: 0.1130 - val_accuracy: 0.9567

 ROC-AUC - epoch: 3 - score: 0.957341

Epoch 00003: val_loss did not improve from 0.11174
Epoch 4/8
1044897/1044897 [==============================] - 103s 99us/step - loss: 0.0940 - accuracy: 0.9617 - val_loss: 0.1115

In [25]:
print("Pred_thresh = ",pred_thresh)
y_test = y_test.reshape((-1, 1)) 
pred_test_y = (y_test>pred_thresh).astype(int) 
submission['prediction'] = pred_test_y 
submission.to_csv("submission.csv", index=False)

Pred_thresh =  0.3
